In [1]:
import pandas as pd

In [2]:
#load the .txt dataset file and verdictfile
#removed .replace("\n", "")
with open('dataset txt files/textfile.txt','r',encoding = "utf-8") as file:
    data = file.read()
with open('dataset txt files/verdictfile.txt','r',encoding = "utf-8") as file2:
    verdicts = file2.read().replace("\n", " ")

In [3]:
textlist = data.split("|\|")
verdictlist = verdicts.split("|\|")

In [4]:
#check if they're the same length
print(len(textlist))
print(len(verdictlist))

23408
23408


In [5]:
#make everything lowercase
for i in range(len(textlist)):
    textlist[i] = textlist[i].lower()
for i in range(len(verdictlist)):
    verdictlist[i] = verdictlist[i].lower()

In [6]:
#load the texts and verdicts into a df
dataset = pd.DataFrame(textlist, columns = ['text'])
dataset["verdict"] = verdictlist

In [7]:
#the code below is to filter out the cases in other languages and cases which are too short and drop those from the df
#https://stackoverflow.com/questions/26697601/how-to-return-elements-from-a-list-that-have-a-certain-length
#https://www.techiedelight.com/find-all-occurrences-item-python-list/
def by_size(words,size):
    result = []
    for index,word in enumerate(words):
        if len(word)<=size:
            result.append(index)
    return result

desired_result = by_size(textlist,2000)
desired_result = list(desired_result)

print(len(desired_result))
print(desired_result[:10])
dataset = dataset.drop(labels=desired_result)
dataset = dataset.reset_index(drop=True)

8479
[3, 7, 13, 19, 27, 39, 40, 48, 49, 50]


In [8]:
print(len(dataset))

14929


In [9]:
#check how many verdicts are still set to "?" and add them to a list
indexlist = []
dummylist = dataset["verdict"].tolist()
for index, i in enumerate(dummylist):
    if i == "?":
        indexlist.append(index)
        
print(len(indexlist))

10464


In [10]:
for i in indexlist:
    if "for these reasons, the court" in dataset.text[i]:
        splitted = dataset.text[i].split("for these reasons, the court",1)
        dataset.text[i] = str(splitted[0])
        dataset.verdict[i] = str(splitted[1])

In [11]:
#how many are still "?"
indexlist2 = []
dummylist = dataset["verdict"].tolist()
for index, i in enumerate(dummylist):
    if i == "?":
        indexlist2.append(index)
print(len(indexlist2))
#drop these from the dataset
dataset = dataset.drop(labels=indexlist2)
dataset = dataset.reset_index(drop=True)

270


In [12]:
#check how many cases are left
len(dataset)

14659

In [13]:
#remove the unwanted parts of each judgment so the model will not get too much information
pure_text = []
for i in range(len(dataset.text)):
    if "the law\n" in dataset.text[i]:
        splitted = dataset.text[i].split("the law\n",1)
        pure_text.append(splitted[0])
    else:
        pure_text.append("d")
    
print(len(pure_text))
print(pure_text.count("d"))

indexlist3 = []
for index, i in enumerate(pure_text):
    if i == "d":
        indexlist3.append(index)
print(len(indexlist3))

for i in range(len(dataset)):
    dataset.text[i] = str(pure_text[i])
    
#drop these from the dataset
dataset = dataset.drop(labels=indexlist3)
dataset = dataset.reset_index(drop=True)

14659
69
69


In [14]:
#filter for cases that are too short again
def by_size(words,size):
    result = []
    for index,word in enumerate(words):
        if len(word)<=size:
            result.append(index)
    return result

dummylist1 = dataset["text"].tolist()

desired_result = by_size(dummylist1,2000)
desired_result = list(desired_result)

print(len(desired_result))
print(desired_result[:10])
dataset = dataset.drop(labels=desired_result)
dataset = dataset.reset_index(drop=True)

688
[24, 29, 30, 31, 32, 33, 36, 38, 39, 59]


In [15]:
#check how many cases are left
print(len(dataset))

13902


In [16]:
#now let's generate numeric labels for each case
#first get rid of the dissenting/concurring opinions
for i in range(len(dataset.verdict)):
    splitted = dataset.verdict[i].split("president",1)
    dataset.verdict[i] = splitted[0]

#then generate labels based on what is in the verdict
binary_verdicts = []
for i in range(len(dataset.verdict)):
    if "there has been no violation" in dataset.verdict[i] and "there has been a violation" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "has not been a violation" in dataset.verdict[i] and "has been a violation" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif " inadmissible" in dataset.verdict[i] and " admissible" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "dismisses" in dataset.verdict[i] and " admissible" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "the application is inadmissible" in dataset.verdict[i] and " admissible" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "there has been a substantive violation" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "there has been a violation" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "there have been violations" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "would amount to a violation" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "disclose a breach of" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "discloses a breach of" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "there was a breach of" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "there has been a breach of" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "state is to pay" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "not be in violation of" in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "would be no violation" in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "would be a violation" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "state has failed to comply with" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif "would be a breach" in dataset.verdict[i]:
        binary_verdicts.append(1)
    elif " dismiss" in dataset.verdict[i] and "violation" not in dataset.verdict[i] and "breach" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "not give rise to a violation" in dataset.verdict[i] and "would give right to a violation" not in dataset.verdict[i]:
        binary_verdicts.append(0)
    elif "decides to strike application" in dataset.verdict[i] and "violation" not in dataset.verdict[i] and "breach" not in dataset.verdict[i]:
        binary_verdicts.append("d")
    elif "decides to strike the case" in dataset.verdict[i] and "violation" not in dataset.verdict[i] and "breach" not in dataset.verdict[i]:
        binary_verdicts.append("d")
    elif "decides to revise the judgment" in dataset.verdict[i] and "violation" not in dataset.verdict[i] and "breach" not in dataset.verdict[i] :
        binary_verdicts.append("d")
    elif "decides to strike the remainder of the application" in dataset.verdict[i] and "violation" not in dataset.verdict[i] and "breach" not in dataset.verdict[i] :
        binary_verdicts.append("d")
    elif "to strike the application" in dataset.verdict[i] and "violation" not in dataset.verdict[i] and "breach" not in dataset.verdict[i] :
        binary_verdicts.append("d")
    else:
        binary_verdicts.append("x")

print(binary_verdicts.count("x"))
print(binary_verdicts.count("d"))
print(binary_verdicts.count(1))
print(binary_verdicts.count(0))

binary_verdicts[17]="f"
binary_verdicts[670]='f'
binary_verdicts[892]='f'
binary_verdicts[1194]='f'
binary_verdicts[1457]='f'
binary_verdicts[1532]='f'
binary_verdicts[1874]='f'
binary_verdicts[3243]='f'
binary_verdicts[3345]='f'
binary_verdicts[3461]='f'

241
570
9542
3549


In [17]:
binary_verdicts.index("x")

575

In [18]:
#drop the faulty cases or for which a label can't be found
for i in range(len(dataset.verdict)):
    dataset.verdict[i] = binary_verdicts[i]

dummylist2 = dataset["verdict"].tolist()
    
indexlist4 = []
for index, i in enumerate(dummylist2):
    if i == "d":
        indexlist4.append(index)
    elif i == "x":
        indexlist4.append(index)
    elif i == "f":
        indexlist4.append(index)
print(len(indexlist4))

dataset = dataset.drop(labels=indexlist4)
dataset = dataset.reset_index(drop=True)

820


In [19]:
print(len(dataset))
dataset.head()

13082


text verdict
0   case of gestur jónsson and ragnar halldór hal...       0
1   case of selahatti̇n demi̇rtaş v. turkey (no. ...       1
2   case of usmanov v. russia\n43936/18   |   jud...       1
3   case of plokhovy v. russia\n45024/07   |   ju...       1
4   case of j.a. v. switzerland\n6325/15   |   ju...       0

In [20]:
import re
#also remove the summary at the start since it contains the outcome of the case
countlist = []
for i in range(len(dataset.text)):
    j = 0
    j += dataset.text[i].count("strasbourg\n")
    if j == 0:
        j += dataset.text[i].count("revision.\n")
    countlist.append(j)
            
print(len(dataset.text))
print(len(countlist))
print(countlist.count(1))

indexes = []
for index, i in enumerate(countlist):
    if i != 1:
        indexes.append(index)

##source for part of code: https://www.iditect.com/how-to/58201148.html
textlist1 = dataset["text"].tolist()
textlist2 = []

for i in range(len(textlist1)):
    if countlist[i] == 1:
        if textlist1[i].count("strasbourg\n") == 1:
            s = textlist1[i]
            res = s[:s.find('legal summary')] + s[s.rfind('strasbourg\n') + 1:].strip()
            textlist2.append(str(res))
        elif textlist1[i].count("revision.\n") == 1:
            s = textlist1[i]
            res = s[:s.find('legal summary')] + s[s.rfind('revision.\n') + 1:].strip()
            textlist2.append(str(res))
    elif " •" not in textlist1[i]:
        textlist2.append(textlist1[i])
    else:
        textlist2.append('d')

print(len(dataset.text))
print(len(textlist2))

print(textlist2.count('d'))

dataset['text'] = textlist2

13082
13082
12101
13082
13082
0


In [21]:
dataset.head()

text verdict
0   case of gestur jónsson and ragnar halldór hal...       0
1   case of selahatti̇n demi̇rtaş v. turkey (no. ...       1
2   case of usmanov v. russia\n43936/18   |   jud...       1
3   case of plokhovy v. russia\n45024/07   |   ju...       1
4   case of j.a. v. switzerland\n6325/15   |   ju...       0

In [22]:
#write the cleaned dataset to new .csv file
dataset.to_csv("cleaned_dataset.csv", header=["text","verdict"])